In [ ]:
from pandas import *
from altair import *
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import accuracy_score

def check_accuracy(*kwargs):
    
    score = accuracy_score(*kwargs)
    
    print(f"The model accuracy is {score:.1%}.")
    
trial = read_csv('data/trial_data.csv')

chart_trial = (
    Chart(trial)
    .mark_point(size=50)
    .encode(
        X("brightness", title="Brightness"),
        Y("points", title="Number of points"),
        Color("up_down:N", title=""),
    )
    .properties(title="Classifying a shape as Up (0) or Down (1)",width=600,height=250)
    .configure_title(anchor="start",fontSize=24)
).configure_axis(
    labelFontSize=16,
    titleFontSize=20
)

# Introduction to Classification using Machine Learning

<img src="imgs/brain_learning.jpg">

<p style="text-align:left;">
    <a href="https://pixabay.com/illustrations/brain-chip-neurons-machine-learning-6010961/">Photo Credit: Pixabay</a>
    <span style="float:right;">
        March 25, 2022 <br>
        Firas Moosvi
    </span>
</p>

## Learning Context (5 mins)

<img src="imgs/avatars.jpg" width=50% align="center">

### Academic program:

- Year 2 of the iSchool Master of Information program
- Concentration: Primarily HCDS students, some UXD, and C&T

### Course Details

- INF 2179 - Machine Learning with Applications in Python
- **Prerequisite**:
    - INF 1340 - Programming for Data Science
- Elective course
- Almost everyone in here wants to be here and is excited to learn more!

### Course Schedule

- Week 1: Review of Python
- Week 2: Loading and cleaning data
- Week 3: Data wrangling
- Week 4: This lesson (Introduction to Classification)

### Programming Experience

- Least experienced: Two terms of working in Python and R sporadically
- Most experienced:  Worked in software industry for 2+ years
- ~ 50 students in the class

### Learning Intentions

- Develop intuition about classification using machine learning.

- Identify the general steps of classification using machine learning.

- Summarize the kNN algorithm and examine its advantages and limitations.

- Critically evaluate the machine learning process and consider the importance of making human-centered choices.

## Overview (1 min)

In today's class we will discuss how to classify data using Machine Learning.

Here is the general algorithm for a machine learning process, for which Classification is a subset:

<img src="https://www.sap.com/dam/application/shared/graphics/what-is-machine-learning-process.svg" width=100%>

**Caption**: General workflow of the Machine Learning process. Image is copyright of [SAP Insights](https://www.sap.com/canada/insights/what-is-machine-learning.html), used under the education copyright exception.

## Introduction to Classification (14 mins)

### Motivation

### Activity


- We are going to see a series of objects, and your task is just to say "up" or "down".

- After each shape, I will tell you the correct answer.

- As we see more, you should get more and more accurate.

<img src="imgs/shape.png" align="center" width=100%>

### Analysis

- What made you decide "up" or "down"?

- What "attributes" did you use to make a decision?

- Which of them were quantitative, and which were qualitative?

| Attribute    | Comment     |
|------------- |------------ |
| Points      |  Number of points           |
| Blob or Pointy | Curved             |
| Brightness             | Scale of 0-100            |
| Color             | red herring, quantify             |

### Debrief

- Can you imagine a computer doing the same task?
- What changes would you make to adapt this task for a machine?
    - Count number of points
    - Quantify the shape
    - Curved or not
    - Colour or Transparency

### Classification Process

Let's revisit the Classification Process in light of the activity.

<img src="https://www.sap.com/dam/application/shared/graphics/what-is-machine-learning-process.svg" width=100%>

**Caption**: General workflow of the Machine Learning process. Image is copyright of [SAP Insights](https://www.sap.com/canada/insights/what-is-machine-learning.html), used under the education copyright exception.

### Load and Visualize Data

In [4]:
# Load data

trial = read_csv('data/trial_data.csv')
trial.head()

,brightness,points,curve,up_down
0,19,0,0,1
1,20,0,0,1
2,21,0,0,1
3,22,0,0,1
4,22,3,0,1


In [5]:
# Visualize Data

chart_trial

alt.Chart(...)

### Classification by Intuition

- We clearly have some intuition about which point belongs to which group. 
- But let's try to formalize this and turn it into an "algorithm"

In [6]:
chart_trial

alt.Chart(...)

### Formalizing our Intuition

- What if we looked at the closest "neighbour" of the test point, and just classify based on the neighbouring point?

- How would we quantify the closest neighbour?
    - Euclidean Distance between point A, $P_A$($x_1$,$y_1$) and point B, $P_B$($x_2$,$y_2$) is:
       - $d = \sqrt{ (x_2 - x_1)^2 + (y_2 - y_1)^2) }$
       
    - Note: Euclidean distance is just the pythagoras theorem!

<img src="imgs/euclidean.png" width=60% align="center">

**Caption**: Definition of the 2D Euclidean distance in relation to the familiar Pythagoras Theorem.

### Setting a Boundary

It becomes clear that if you were to check every point with the classifier, a boundary forms.
Here's what that looks like with the current classifier and the current training data:

<img src="imgs/boundary.png" width=60%>

Quite often, decision boundaries are not that clear (otherwise we wouldn't need machine learning!) and the more data (and classes) there are, the more likely it is that boundaries are complex!

## $k$ Nearest Neighbours (kNN) method (8 mins)

<img src ="imgs/knn.png" width=60%>

**Caption**: Pictoral description of the kNN algorithm, with three different classes (A, B, C) and a test point ($P_t$). The $k$ nearest neighbours of $P_t$ dictates which class the point belongs to. Source: {cite}`Atallah:2019`

### Problem 1: Scaling

- What is one problem with this method?
    - Points that look equidistant will not have the same distances unless the data are re-scaled
    

- **Solution: Re-scale the data!**
    
    - To address this problem, we can re-scale the data by transforming the data so it has a mean of 0, and a standard deviation of 1.

    - We can do this by subtracting the mean $\bar{x}$, and then dividing by the standard deviation $\sigma$.

    $$
    Data_{scaled} = \frac{x - \bar{x}}{\sigma}
    $$

    - You may recognize this as a $z$-score

### Problem 2: Stability/outliers

- What is a second problem with this method?
    - If our data is noisy, this method is prone to errors due to outliers and/or just random chance

- **Solution: Consider more than one neighbour!**
    - How many neighbours?
        - Choose an odd number so there are no ties!
        - Start with a low-number ($k_1 = 3$), run your classification on the training dataset, record accuracy; repeat for a bunch of $k$ values.
        - Plot the accuracy vs. $k$, and pick the highest one! (Yes, really!)
        - Rough guideline: $k$ will depend on the size of your dataset. Probably not worth it to increase $k$ beyond $\approx \sqrt{N}$, where $N$ is the number of data points.

### Demo of kNN

Steps of kNN:

- Load Training Data
- Visualize Data
- Process Data
- Create classifier
- Train Algorithm
- Check accuracy of classifier

In [7]:
# First let's load our data set
train = read_csv('data/train.csv')
train

,brightness,points,curve,up_down
0,28,8,0,1
1,23,1,1,0
2,24,3,0,1
3,20,0,0,1
4,27,3,0,0
...,...,...,...,...
4217,27,1,0,0
4218,50,3,1,1
4219,43,1,0,1
4220,44,1,0,0


That's a lot of data points! 

Let's try to visualize it:

<img src="imgs/train.png" width=80%>

That's a lot of points! It's probably not going to be so easy to easily visualize a decision boundary on this one.

Let's go ahead and set up our Machine Learning problem!

In [8]:
train

,brightness,points,curve,up_down
0,28,8,0,1
1,23,1,1,0
2,24,3,0,1
3,20,0,0,1
4,27,3,0,0
...,...,...,...,...
4217,27,1,0,0
4218,50,3,1,1
4219,43,1,0,1
4220,44,1,0,0


In [9]:
# First, let's rescale our data set and keep the means and standard deviations

train_mean = train.mean().to_dict()
train_std = train.std(ddof=0).to_dict()

for col in ['brightness','points']:
    train[col] = (train[col] - train_mean[col]) / train_std[col]
    
train.head()

,brightness,points,curve,up_down
0,-0.548883,0.913568,0,1
1,-0.978713,-0.508361,1,0
2,-0.892747,-0.102096,0,1
3,-1.236610,-0.711494,0,1
4,-0.634849,-0.102096,0,0


In [10]:
# Then, let's create a classifier 

classifier = KNN(n_neighbors=5, metric='euclidean')

In [11]:
# Now, let's Train our model

classifier.fit(X = train[['brightness','points','curve']],
               y = train['up_down'])

KNeighborsClassifier(metric='euclidean')

In [12]:
# Let's use the classifier to predict each row of our training data set:

predictions = classifier.predict(train[['brightness','points','curve']])
predictions

array([1, 0, 1, ..., 0, 0, 0])

Slightly anticlimatic right?

Let's check to see how well we did...

### Review of KNN

<img src="https://cambridgecoding.files.wordpress.com/2016/01/knn2.jpg" width=100%>

[Source: Natasha Latysheva, Cambridge Coding Academy](https://cambridgecoding.wordpress.com/2016/01/16/machine-learning-under-the-hood-writing-your-own-k-nearest-neighbour-algorithm/)

## Checking prediction accuracy (5 mins)

- How do we check how well we did?

In [14]:
check_accuracy(train['up_down'],predictions) 

The model accuracy is 70.5%.


At this point you should question your decisions.

<img src="https://media3.giphy.com/media/8cPKrvyEpAMu2QFDWe/giphy.gif?cid=ecf05e47v152n2xlz4w3weap7880dy81u4rq4hs1zlds4o2q&rid=giphy.gif&ct=g">

### Separating the training and testing data

- To determine accuracy, you **MUST** use data that the classifier has never seen before.

- Some people call this, the "Golden Rule" of Machine Learning:

> The Test data cannot influence the training data in any way"

### Training and Testing Paradigm

- Before you start working with the data, you should "split" the data into a "training set" and a "test set"

<img src="imgs/training_test.png" width=80%>

- Luckily, I did all the split for you and saved it as a separate file: `test.csv`

In [16]:
# Let's load the test data set

test = read_csv('data/test.csv')
test

,brightness,points,curve,up_down
0,26,0,1,0
1,22,3,1,1
2,57,0,0,0
3,34,1,0,0
4,21,1,0,0
...,...,...,...,...
1051,33,14,0,1
1052,22,0,1,1
1053,42,0,0,0
1054,44,6,0,1


In [17]:
# First, let's rescale our data the *exact* same we did our training data set

for col in ['brightness','points','curve']:
    test[col] = (test[col] - train_mean[col]) / train_std[col]

In [18]:
# Let's use the classifier to predict each row of our TESTING data set:

predictions = classifier.predict(test[['brightness','points','curve']])
predictions

array([0, 1, 0, ..., 0, 1, 1])

In [20]:
# And finally, check our accuracy
check_accuracy(test['up_down'],predictions)

The model accuracy is 64.3%.


## Context Matters (10 mins)

- Let's start with the accuracy numbers:
    - Our algorithm gets it right ~ 63 times out of 100.
    - That's not bad - it's okay if we mis-classify some of them right?

- What if I told you that the data you've been working with has nothing to do with shapes, but people?

<img src="https://media3.giphy.com/media/8cPKrvyEpAMu2QFDWe/giphy.gif?cid=ecf05e47v152n2xlz4w3weap7880dy81u4rq4hs1zlds4o2q&rid=giphy.gif&ct=g" align="center">

### The Context

<img src="https://www.pewresearch.org/wp-content/uploads/2021/08/FT_21.08.12_Incarceration_2.png" width=100%>

[Source: Pew Research](https://www.pewresearch.org/fact-tank/2021/08/16/americas-incarceration-rate-lowest-since-1995/)

- It is illegal in the United States for judges to sentence based on race, color, religion, sex, age.

- And yet...

<img src="https://www.sentencingproject.org/wp-content/uploads/2021/05/lifetime-likelihood-by-race.png" width=100%>

[Source: The Sentencing Project](https://www.sentencingproject.org/criminal-justice-facts/)

### Enter Machine Learning... 

Motivation

> "The United States locks up far more people than any other country, a disproportionate number of them black. For more than two centuries, the key decisions in the legal process, from pretrial release to sentencing to parole, have been in the hands of human beings guided by their instincts and personal biases."

- [Machine Bias Risk Assessments in Criminal Sentencing](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing)




Risk Scores

> "If computers could accurately predict which defendants were likely to commit new crimes, the criminal justice system could be fairer and more selective about who is incarcerated and for how long. The trick, of course, is to make sure the computer gets it right. If it’s wrong in one direction, a dangerous criminal could go free. If it’s wrong in another direction, it could result in someone unfairly receiving a harsher sentence or waiting longer for parole than is appropriate."

- [Machine Bias Risk Assessments in Criminal Sentencing](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing)

COMPAS is one such proprietary software, developed by a for-profit company called Northpointe.

Here's how it performs:

<img src="imgs/propublica2.png" width=80%>

[Source: ProPublica](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing)

### Coming Clean

<img src="imgs/propublica.png" width=100%>

[Source: ProPublica](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing)

<img src="imgs/shape_context.png" width=100%>

[Source: ProPublica](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing)

- Here's what the data actually means, all I did was change the names of the columns:

| Current Column Name | Actual Column Name |
|---------------------|--------------------|
| Number of Points    | Prior Arrests                   |
| Brightness          | Age                   |
| Curve               | Sex                   |
| Up_Down             | Re-offense committed (recidivism)                   |

- Now let's re-evaluate the algorithm performance (63%) with this contextual information.

- When we changed the context, we started to look beyond numbers, and saw human beings instead.


- Consider your choices carefully - they may seem inoccuous initially, but hiding behind black box algorithms and deploying weapons of Math Destruction without appropriately scrutinizing the underlying data is DANGEROUS!



- "Data" is not a panacea or a magical cure-all ; it's not biased in itself but it **does** reflect the biases, oppressions, systemic inequties within our society.

## Preview of Next week (1 min)

Over the next few weeks in lectures and labs we will:

- carefully reproduce the Propublica analysis by treating the Northpointe algorithm as a binary classifier.

- analyze what it means for an algorithm to be "fair".

- look at Northpointe's response, and those of some other academics about this issue.

- consider metrics beyond simple "accuracy".

- learn some more sophisticated ML algorithms to see if we can remove bias (**Spoiler Alert: We will fail spectacularly!**)

## Summary (5 mins)

- Intuition on classification with Machine Learning
- Summary of kNN method
- Checking the accuracy of our classifier
- Importance of Context

<img src="https://www.sap.com/dam/application/shared/graphics/what-is-machine-learning-process.svg" width=100%>

**Caption**: General workflow of the Machine Learning process. Image is copyright of [SAP Insights](https://www.sap.com/canada/insights/what-is-machine-learning.html), used under the education copyright exception.

<img src ="imgs/knn.png" width=60%>

**Caption**: Pictoral description of the kNN algorithm, with three different classes (A, B, C) and a test point ($P_t$). The $k$ nearest neighbours of $P_t$ dictates which class the point belongs to. Source: {cite}`Atallah:2019`

## Summary of Learning Intentions

- Develop intuition about classification using machine learning.

- Identify the general steps of classification using machine learning.

- Summarize the kNN algorithm and examine its advantages and limitations.

- Critically evaluate the machine learning process and consider the importance of making human-centered choices.

## Activity: You Try (homework)

- Task 1: Use the same training data and increase the number of "neighbours", or the $k$ value.

- Task 2: Compute performance metrics of the new prediction.

- Task 3: Write a loop to do this classification for all $k$ values from 1 to $N$ where $N$ is the total number of data points.
    - Plot the accuracy vs. $N$; for which $k$ value is the accuracy highest?

We will briefly review these tasks at the start of next class!

### Acknowledgements

- UC Berkeley Foundations of Data Science Data8 course notes, in particular [David Wagner's chapter on Classification](https://inferentialthinking.com/chapters/17/Classification.html).

- [ProPublica article](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing) that inspired this class.

- [Allen Downey's Recidivism Case Study](https://allendowney.github.io/RecidivismCaseStudy/index.html).

- UBC's [Mike Gelbart's video lectures on KNN](https://www.youtube.com/watch?v=JRF6oELLn0M&list=PLWmXHcz_53Q02ZLeAxigki1JZFfCO6M-b&index=4).

### Extending kNN

- We've only talked about two different classes (up or down) with just two input variables (Number of points and Brightness).

- It's a bit harder to imagine in multiple dimensions, but here is what the picture looks like in 3 dimensions:

<img src="https://inferentialthinking.com/_images/Implementing_the_Classifier_12_0.png" width=60%>

**Caption**: Visualization of points in 3-dimensional space with three input variables and two classes. Source: [Computational and Inferential Thinking: The Foundations of Data Science](https://inferentialthinking.com/chapters/intro.html) by Ani Adhikari, John DeNero, David Wagner distributed under a CC BY-NC-ND 4.0 license.

- To calculate the Euclidean distance between two points in three dimensions with points , we just extend the 2D formula.

- For two points in 3D space, $P_C$($x_1$,$y_1$,$z_1$) and $P_D$($x_2$,$y_2$,$z_2$):

    - $d = \sqrt{ (x_2 - x_1)^2 + (y_2 - y_1)^2 + (z_2 - z_1)^2}$


- And so on... for points in N-dimensions